<a href="https://colab.research.google.com/github/LucasRios/AIPhytonExamples/blob/main/m%C3%A9tricas_e_avalia%C3%A7%C3%A3o_de_aprendizado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, optimizers
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix

In [4]:
# Download do dataset
!wget --no-check-certificate https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip

#extração
local_zip = '/content/kagglecatsanddogs_5340.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

#limpeza de imagens corrompidas
num_skipped = 0
for folder_name in ("PetImages/Cat", "PetImages/Dog"):
  folder_path = os.path.join("/content", folder_name)
  for fname in os.listdir(folder_path):
    fpath = os.path.join(folder_path, fname)
    try:
        fobj = open(fpath, "rb")
        is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
    finally:
        fobj.close()
    if not is_jfif:
        num_skipped += 1
        os.remove(fpath)
print("Imagens corrompidas removidas: %d" % num_skipped)


--2025-12-26 18:09:26--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.63.106.22, 2600:1408:c400:f95::317f, 2600:1408:c400:f86::317f
Connecting to download.microsoft.com (download.microsoft.com)|23.63.106.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824887076 (787M) [application/octet-stream]
Saving to: ‘kagglecatsanddogs_5340.zip’

kagglecatsanddogs_5 100%[===================>] 786.67M  93.7MB/s    in 9.0s    

2025-12-26 18:09:35 (87.2 MB/s) - ‘kagglecatsanddogs_5340.zip’ saved [824887076/824887076]

Imagens corrompidas removidas: 1590


In [5]:
#Preparação dos Dados (Data Generators)
base_dir = 'PetImages'
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2) # 20% para teste

train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(224,224),
    batch_size=30,
    class_mode='binary',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='binary',
    subset='validation',
    shuffle=False
)


Found 18729 images belonging to 2 classes.
Found 4681 images belonging to 2 classes.


In [6]:
#Construindo o Modelo com Transfer Learning
vgg_base = tf.keras.applications.VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(224,224,3)
)

vgg_base.trainable = False

model = models.Sequential([
    vgg_base, layers.Flatten(), layers.Dense(256, activation='relu'), layers.Dropout(0.5), layers.Dense(1, activation='sigmoid')

])

model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])

model.summary()

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     6,422,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,137,729 (80.63 MB)

 Trainable params: 6,423,041 (24.50 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [7]:
#Treinamento
history = model.fit(train_generator, epochs=2, validation_data=validation_generator)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
  4/625 ━━━━━━━━━━━━━━━━━━━━ 3:04:16 18s/step - accuracy: 0.5507 - loss: 0.7266

KeyboardInterrupt: 

In [9]:
## 1. Gerar previsões
Y_pred = model.predict(validation_generator)
y_pred = (Y_pred > 0.5).astype(int).ravel()
y_true = validation_generator.classes
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))

# 2. Obter a Matriz de Confusão
# No scikit-learn: TN, FP, FN, TP
tn,fp,fn,tp = confusion_matrix(y_true, y_pred).ravel()

# 3. Funções das Métricas (Implementação Manual)
def mostrar_metricas(vp,vn,fp,fn):
    print('Taxa de Verdadeiro Positivo: ', vp)
    print('Taxa de Verdadeiro Negativo: ', vn)
    print('Taxa de Falso Positivo: ', fp)
    print('Taxa de Falso Negativo: ', fn)

    acuracia = (vp + vn) / (vp + vn + fp + fn)
    print('Acurácia: ', acuracia)

    precisao = vp / (vp + fp)
    print('Precisão: ', precisao)

    sensibilidade = vp / (vp + fn)
    print('Sensibilidade: ', sensibilidade)

    especificidade = vn / (vn + fp)
    print('Especificidade: ', especificidade)

    f_score = 2 * (precisao * sensibilidade) / (precisao + sensibilidade)
    print('F-Score: ', f_score)

# Exibir resultados reais do modelo
mostrar_metricas(tp, tn, fp, fn)

  5/147 ━━━━━━━━━━━━━━━━━━━━ 53:51 23s/step

KeyboardInterrupt: 